Requirements
For Project 3, you will work with your group to tell a story using data visualizations. Here are the specific requirements:

Your visualization must include a Python Flask-powered API, HTML/CSS, JavaScript, and at least one database (SQL, MongoDB, SQLite, etc.).

Your project should fall into one of the following three tracks:

A combination of web scraping and Leaflet or Plotly

A dashboard page with multiple charts that update from the same data

A server that performs multiple manipulations on data in a database prior to visualization (must be approved)

Your project should include at least one JS library that we did not cover.

Your project must be powered by a dataset with at least 100 records.

Your project must include some level of user-driven interaction (e.g., menus, dropdowns, textboxes).

Your final visualization should ideally include at least three views.

In [1]:
#all dependecias imports
import pprint
from amadeus import Client, ResponseError
import logging
from amadeus_keys import client_id, client_secret
from flask import Flask, jsonify, render_template
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from bs4 import BeautifulSoup
import json
import requests


In [2]:
amadeus = Client(
    client_id=client_id,
    client_secret=client_secret
)
locations = [
    {'latitude': 40.7128, 'longitude': -74.0060, 'name': 'New York City'},
    {'latitude': 13.023577, 'longitude': 77.642256, 'name': 'Bangalore'},
    {'latitude': 41.42, 'longitude': 2.228208, 'name': 'Barcelona'},
    {'latitude': 52.541755, 'longitude': 13.457198, 'name': 'Berlin'},
    {'latitude': 32.806993, 'longitude': -96.737293, 'name': 'Dallas'},
    {'latitude': 51.520180, 'longitude': -0.061048, 'name': 'London'},
    {'latitude': 48.91, 'longitude': 2.46, 'name': 'Paris'},
    {'latitude': 37.810980, 'longitude': -122.370076, 'name': 'San Francisco'}
]
try:
    for location in locations:
        response = amadeus.reference_data.locations.points_of_interest.get(
            latitude=location['latitude'],
            longitude=location['longitude']
        )
        points_of_interest = response.data
        print(f"Points of interest for {location['name']}:")
        pprint.pprint(points_of_interest)
        print()
except ResponseError as error:
    print(error.response.body)
    raise error


            {
                "errors": [
                        {
                            "code": 38189,
                            "title": "Internal error",
                            "detail": "An internal error occurred, please contact your administrator",
                            "status": 500
                        }
                ]
            }
        


ServerError: [500]


In [ ]:
category = locations[0]
category

In [ ]:
amadeus = Client(
    client_id=client_id,
    client_secret=client_secret
)
locations = [
    {'latitude': 40.7128, 'longitude': -74.0060, 'name': 'New York City'},
    {'latitude': 13.023577, 'longitude': 77.642256, 'name': 'Bangalore'},
    {'latitude': 41.42, 'longitude': 2.228208, 'name': 'Barcelona'},
    {'latitude': 32.806993, 'longitude': -96.737293, 'name': 'Dallas'},
    {'latitude': 51.520180, 'longitude': -0.061048, 'name': 'London'},
    {'latitude': 48.91, 'longitude': 2.46, 'name': 'Paris'}
]
# Initialize an empty list to store the points of interest
poi_list = []

try:
    for location in locations:
        response = amadeus.reference_data.locations.points_of_interest.get(
            latitude=location['latitude'],
            longitude=location['longitude']
        )
        points_of_interest = response.data
        for poi in points_of_interest:
            # Extract the desired information from each point of interest
            poi_info = {
                'city': location['name'],
                'category': poi['category'],
                'name': poi['name'],
                'rank': poi['rank'],
                           }
            poi_list.append(poi_info)

except ResponseError as error:
    print(error.response.body)
    raise error

# Create a DataFrame from the list of points of interest
df = pd.DataFrame(poi_list)

# Print the DataFrame
print(df)

select cities: NYC, LA, SF, Miami, Boston, DC
travel plans: weather, hotels, attractions, transportation

In [ ]:
from config import geoapify_key
import json
import requests

In [ ]:
import requests
import pandas as pd
import ipywidgets as widgets

# Create a dropdown widget for city selection
city_dropdown = widgets.Dropdown(
    options=['New York City', 'Los Angeles','Mount Vernon','Yonkers'],
    value='Yonkers',
    description='City:'
)

# Display the dropdown widget
city_dropdown

# Set the geographical coordinates based on the selected city
city_coordinates = {
    'New York City' : {'latitude': 40.7128, 'longitude': -74.0060},
    'Mount Vernon' : {'latitude': 40.9136005, 'longitude': -73.8290816},
    'White Plains' : {'latitude': 40.033985, 'longitude': -73.762909},
    'Yonkers' : {'latitude': 40.9025, 'longitude': -73.9247}
    }

# Function to handle the dropdown value change
def handle_dropdown_change(change):
    if change['name'] == 'value':
        selected_city = change['new']
        latitude = city_coordinates[selected_city]['latitude']
        longitude = city_coordinates[selected_city]['longitude']

        # Set the parameters for the type of place
        categories = "catering.restaurant"
        conditions = "vegetarian"
        radius = 8000

        # Set the parameters for the type of search
        filters = f"circle:{longitude},{latitude},{radius}"
        bias = f"proximity:{longitude},{latitude}"
        limit = 20

        # Set up a parameters dictionary
        params = {
            "categories": categories,
            "conditions": conditions,
            "limit": limit,
            "filter": filters,
            "bias": bias,
            "apiKey": geoapify_key
        }

        # Set base URL
        base_url = "https://api.geoapify.com/v2/places"

        # Run a request using our params
        response = requests.get(base_url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            features = data['features']
            properties_list = []

            # Extract the required properties from each feature
            for feature in features:
                properties = feature['properties']
                properties_list.append(properties)

            # Create a DataFrame from the properties list
            df = pd.DataFrame(properties_list)
            print(df)
        else:
            print("API request failed.")

# Register the handle_dropdown_change function to handle value changes
city_dropdown.observe(handle_dropdown_change)

# Display the dropdown widget
city_dropdown


In [ ]:
import requests
import ipywidgets as widgets

# Set the parameters for the geocode request
params = {
    "filter": "city",
    "apikey" = geoapify_key,
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

# Make the request to Geoapify API
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    cities = []

    # Extract city names from the response
    for feature in data["features"]:
        properties = feature["properties"]
        cities.append(properties["city"])

    # Create a dropdown widget for city selection
    city_dropdown = widgets.Dropdown(
        options=cities,
        description="City:"
    )

    # Display the dropdown widget
    city_dropdown
else:
    print("API request failed.")


In [ ]:
# update params dict
target_city = "Sydney Opera House"

params = {
    "text": target_city,
    "apiKey": geoapify_key
}

# Build URL using the geocode endpoint
base_url = "https://api.geoapify.com/v1/geocode/search"

In [ ]:
# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
# Extract lat/lon
lat = response["features"][0]["properties"]["lat"]
lon = response["features"][0]["properties"]["lon"]

# Print results
print(f"{target_city}: {lat}, {lon}")

In [ ]:
# Set the geographical coordinates for Sydney, Australia
#{'latitude': 40.7128, 'longitude': -74.0060, 'name': 'New York City'},
latitude = 40.7128
longitude = -74.0060

# Set the parameters for the type of place
categories = "catering.restaurant"
conditions = "vegetarian"
radius = 8000

# Set the parameters for the type of search
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 20

# set up a parameters dictionary
params = {
    "categories":categories,
    "conditions":conditions,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [ ]:
print(response.url)

In [ ]:
 # convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    features = data['features']
    properties_list = []

 

In [ ]:
# Extract the required properties from each feature
for feature in features:
    properties = feature['properties']
    properties_list.append(properties)

    # Create a DataFrame from the properties list
    df = pd.DataFrame(properties_list)
    print(df)
else:
    print("API request failed.")